# Filter posts on the reddit related to the 2016 U.S. presidential election

python=3.7.16

2016 POTUS Election（time：October 10 to December 19, 2016)

All data sets analysis

In [1]:
import pandas as pd
from joblib import Parallel, delayed  # parallel computing
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore')

## Filter debunking data

In [ ]:
# read the data sets one by one and append them together
import glob  
  
# file path contains all the files of reddit comment data
folder_path = '/mnt/data/wenlu/reddit-new/16-election-comment'  
  
# use glob library to find all the csv files
csv_files = glob.glob(f'{folder_path}/*.csv')  
  
# empty DataFrame to save all the data
all_data = pd.DataFrame()  

for file in csv_files:  
    df = pd.read_csv(file)  
    all_data = all_data.append(df, ignore_index=True)  
print(all_data)

In [4]:
%%time 
#all_data = pd.read_csv('all_data_16election.csv')
all_data.dropna(subset=['body'])  # drop the rows of 'body' columns containing NA / NaN value  

CPU times: user 4min 43s, sys: 44.3 s, total: 5min 28s
Wall time: 5min 28s


,Unnamed: 0,subreddit_id,created_utc,subreddit,author,id,parent_id,body,lang
0,0,t5_3b66u,1.477958e+09,furry_irl,u/PM_ME_YIFF_PICS,d9g52fi,t1_d9fqrn9,yes I am,tr
1,1,t5_2qh03,1.477958e+09,gaming,u/PhyrexStrike,d9g52fj,t1_d9ft6iq,[I definitely had flashbacks to that when that...,en
2,2,t5_3hauq,1.477958e+09,ResearchSource,u/Psychonaut76,d9g52fk,t3_5aep3k,Nice review. How long from order to door was it?,en
3,3,t5_2t1bl,1.477958e+09,Rainbow6,u/Oxi_,d9g52fl,t1_d9g1wcq,"It's a fairly big issues lol, I made this post...",en
4,4,t5_2uu0j,1.477958e+09,paragon,u/catdeuce,d9g52fm,t1_d9g45us,I have it.,en
...,...,...,...,...,...,...,...,...,...
78751994,71022314,t5_2qlqq,1.480550e+09,fantasyfootball,u/TheImmatureLawyer,damufh6,t1_damex80,You are right from a team perspective. But if ...,en
78751995,71022315,t5_2ra29,1.480550e+09,buffalobills,u/ICookTheBlueStuff,damufh7,t1_damud1z,Missing the word: corner.\n\nYeah I dont see A...,en
78751996,71022316,t5_2qh1i,1.480550e+09,AskReddit,u/jflb96,damufh8,t1_daml04y,"But do it quick, because Outrageous_Claims' da...",en
78751997,71022317,t5_2t1bl,1.480550e+09,Rainbow6,u/andlkam2,damufh9,t3_5fqsmc,Poland's counter is Blitz.,en


# Topic & debunking keywords

In [13]:
# German
keywords_German = ["Trumpf", "Machen Sie Amerika wieder großartig", "Parlamentswahl 2016", "Amerika zuerst", "Hillary", "MachenSieAmerikawiedergroßartig", "Ich bin mit ihr zusammen", "Debatten2016", "Wahlen2016", "Clinton"]

# French
keywords_French = ["MAGA", "Rendre l'Amérique encore une fois formidable", "Trump", "L'Amérique d'abord", "Rendre sa grandeur à l’Amérique", "Hillary", "Clinton", "Je suis avec elle", "OHHillYes","élection 2016", "élection 2016", "débats 2016"]

# topic key words filtering
keywords_English = ["MAGA", "MakeAmericaGreatAgain", "Trump", "AmericaFirst", "Make America Great Again", \
             "Hillary", "Clinton", "ImWithHer", "OHHillYes", \
             "election2016", "elections2016", "debates2016"]

In [8]:
db_df = pd.read_csv("/mnt/data/wenlu/new—program/data/(Merge_ver3)debunking_fact-checking_sites.csv") # get the debunking media information
db_df = db_df.dropna(subset=['domain'])
db_df_domain = db_df['domain'].to_list()
db_df_name = db_df['name'].to_list() # get more debunking key words

In [12]:
# add some Germany debunking media
db_german_media = ['rumoursaboutgermany.info','faz.net', 'welt.de', 'correctiv.org','tagesschau.de/faktenfinder','zdf-investigativ','deutschlandfunk.de/']
# topic key words filtering
db_keywords_English = ["fact check", "fact-checking", "fact checker", "fact checkers", "fake news", "misinformation", "disinformation", "debunkers", "debunker", "debunking", "debunking", "debunk"]
db_keywords_German = ['Faktencheck',"Faktenprüfung", "Faktenprüfer","faktenprüfer", "gefälschte Nachrichten","Fehlinformation", "fehlinformation","Desinformation", "desinformation","Entlarver", "Entlarven", "entlarven", "entlarver", "entlarven"]
db_keywords_French = ['vérification des faits','fausses nouvelles','désinformation','fausse information','Réfuter les rumeurs','briseur de rumeur','discréditer']

# add the debunk domain
db_keyword_English_contains_domain = (keywords_English + db_df_domain + db_df_name)
db_keyword_Ge_contains_domain = (db_keywords_German + db_df_domain + db_df_name + db_german_media)
db_keyword_Fr_contains_domain = (db_keywords_French + db_df_domain + db_df_name)

# English data analysis

In [ ]:
# the dataframe"all_data" contains the language info(after langdetect)
df_en = all_data[all_data['lang'] == 'en']
df_en.to_csv('16election[lang=en][time=16.10-16.12].csv')

In [6]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59184212 entries, 1 to 78751998
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    object 
 1   subreddit_id  object 
 2   created_utc   float64
 3   subreddit     object 
 4   author        object 
 5   id            object 
 6   parent_id     object 
 7   body          object 
 8   lang          object 
dtypes: float64(1), object(8)
memory usage: 4.4+ GB


In [14]:
%%time
df_en_topic_key = df_en[df_en['body'].str.contains('|'.join(keywords_English), case=False)]  # keywords searching in the body text
df_en_topic_key.sort_index(axis=0, ascending=True, inplace=True)  # restore posting order
df_en_topic_key.drop_duplicates(subset='id', keep='first', inplace=True)  # drop duplicates
dump(df_en_topic_key,"16-pkl/16election_[lang=en][time=16.10-16.12].pkl")

CPU times: user 27min 29s, sys: 7.51 s, total: 27min 36s
Wall time: 27min 37s


['16-pkl/16election_[lang=en][time=16.10-16.12].pkl']

In [15]:
df_en_topic_key.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2060294 entries, 24 to 78751988
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    object 
 1   subreddit_id  object 
 2   created_utc   float64
 3   subreddit     object 
 4   author        object 
 5   id            object 
 6   parent_id     object 
 7   body          object 
 8   lang          object 
dtypes: float64(1), object(8)
memory usage: 157.2+ MB


In [16]:
%%time
# debunk keywords filt
df_en_db_key = df_en_topic_key[df_en_topic_key['body'].str.contains('|'.join(db_keyword_English_contains_domain), case=False)]  # keywords searching in the body text
df_en_db_key.sort_index(axis=0, ascending=True, inplace=True)  # Restore posting order
df_en_db_key.drop_duplicates(subset='id', keep='first', inplace=True)  # drop duplicates
dump(df_en_db_key,"16-pkl/16election_debunk_[lang=en][time=16.10-16.12].pkl")

CPU times: user 15min 9s, sys: 890 ms, total: 15min 10s
Wall time: 15min 10s


['16-pkl/16election_debunk_[lang=en][time=16.10-16.12].pkl']

In [17]:
df_en_db_key.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30297 entries, 6793 to 78746321
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    30297 non-null  object 
 1   subreddit_id  30297 non-null  object 
 2   created_utc   30297 non-null  float64
 3   subreddit     30297 non-null  object 
 4   author        30297 non-null  object 
 5   id            30297 non-null  object 
 6   parent_id     30297 non-null  object 
 7   body          30297 non-null  object 
 8   lang          30297 non-null  object 
dtypes: float64(1), object(8)
memory usage: 2.3+ MB


## specific time filtering
set time window to specific time eg.October 10 to December 19, 2016

In [24]:
all_data['created_utc']

0           1477958400
1           1477958400
2           1477958400
3           1477958400
4           1477958400
               ...    
71022314    1480550399
71022315    1480550399
71022316    1480550399
71022317    1480550399
71022318    1480550399
Name: created_utc, Length: 71022319, dtype: int64

In [25]:
# Oct. 10, 2016's utc timestamp:1476028801
# Convert the string timestamp to an integer
timestamp_oct_10_2016 = 1476028801

# Boolean mask for filtering time
mask = (all_data['created_utc'] >= timestamp_oct_10_2016)

# Apply the mask to the DataFrame
df_t = all_data[mask]

In [26]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71022319 entries, 0 to 71022318
Data columns (total 17 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   subreddit_id            object
 1   link_id                 object
 2   created_utc             int64 
 3   subreddit               object
 4   retrieved_on            int64 
 5   stickied                bool  
 6   author_flair_text       object
 7   score                   int64 
 8   controversiality        int64 
 9   author                  object
 10  edited                  object
 11  distinguished           object
 12  id                      object
 13  gilded                  int64 
 14  author_flair_css_class  object
 15  parent_id               object
 16  body                    object
dtypes: bool(1), int64(5), object(11)
memory usage: 9.1+ GB


In [27]:
df_t = df_t.sort_values(by='created_utc').reset_index(drop=True)

In [28]:
df_t.to_csv('new_16election_reddit_comment[time=16.10.10-16.12.31].csv')